# Препроцессинг и обучение модели

## Загрузка данных и подготовка среды для работы

In [1]:

! pip install transformers datasets accelerate evaluate

In [2]:
import os
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import pandas as pd
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split


import evaluate
import numpy as np

from transformers import pipeline



In [3]:
for dirpath, _, filenames in os.walk('/content/category_classification/'):
  for filename in filenames:
    print(os.path.join(dirpath, filename))

/content/category_classification/table_for_category_classification.csv
/content/category_classification/runs/Apr11_07-47-37_36950bd5f6f5/events.out.tfevents.1712821658.36950bd5f6f5.501.1
/content/category_classification/runs/Apr11_07-43-19_36950bd5f6f5/events.out.tfevents.1712821399.36950bd5f6f5.501.0


## Конфигурация

In [10]:


class Config:
    DATASET_ID = "/content/category_classification/table_for_category_classification.csv"
    MODEL_CKPT = "distilbert-base-uncased"
    SRC_COLUMN = "Название"
    TGT_COLUMN = "Категория"
    TEST_SIZE = 0.2
    SEED = 0
    MAX_LEN = 32
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    ID2LABEL = {0:'Прочее', 1:'Инструментальная диагностика', 2:'Манипуляция',
                3:'Консультация специалиста', 4:'Лабораторная диагностика'}
    LABEL2ID = {'Прочее':0, 'Инструментальная диагностика':1, 'Манипуляция':2,
                'Консультация специалиста':3, 'Лабораторная диагностика':4}
    EVAL_METRIC = "accuracy"
    MODEL_OUT_DIR = "category_classification"
    NUM_EPOCHS = 3
    LR = 2E-5
    BATCH_SIZE = 16
    WEIGHT_DECAY = 0.01
    EVAL_STRATEGY = "epoch"
    SAVE_STRATEGY = "epoch"
    LOGGING_STRATEGY = "epoch"
    #PUSH_TO_HUB = False
    PUSH_TO_HUB = True

config = Config()

## Предобработка данных

In [5]:

class TextClassificationDataset:
    def __init__(self):
        self.dataset_id = config.DATASET_ID
        self.model_ckpt = config.MODEL_CKPT
        self.src_column = config.SRC_COLUMN
        self.tgt_column = config.TGT_COLUMN
        self.test_size = config.TEST_SIZE
        self.seed = config.SEED
        self.max_len = config.MAX_LEN
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_ckpt)

    def create_data(self):
        self.df =pd.read_csv(self.dataset_id, sep=',', on_bad_lines='skip')
        self.df = self.df[[self.src_column, self.tgt_column]]
        self.df[self.src_column] = self.df[self.src_column].apply(lambda x: x.lower())
        self.df[self.tgt_column] = self.df[self.tgt_column].apply(lambda x: 2 if x==-1 else x)
        self.df = self.df.drop_duplicates()
        level_map = {'Прочее':0, 'Инструментальная диагностика':1, 'Манипуляция':2,
                     'Консультация специалиста':3, 'Лабораторная диагностика':4}
        self.df[self.tgt_column] = self.df[self.tgt_column].map(level_map)
        self.train_df, self.test_df = train_test_split(self.df, test_size=self.test_size, shuffle=True, random_state=self.seed, stratify=self.df[self.tgt_column])
        self.train_data = Dataset.from_pandas(self.train_df)
        self.test_data = Dataset.from_pandas(self.test_df)
        return self.train_data, self.test_data

    def tokenize_function(self, example):
        model_inp = self.tokenizer(example[self.src_column], truncation=True, padding=True, max_length=self.max_len)
        labels = torch.tensor(example[self.tgt_column], dtype=torch.int)
        model_inp["labels"] = labels
        return model_inp

    def preprocess_function(self, data):
        model_inp = data.map(self.tokenize_function, batched=True, remove_columns=data.column_names)
        return model_inp

    def gen_classification_dataset(self):
        train_data, test_data = self.create_data()
        train_tokenized_data = self.preprocess_function(train_data)
        test_tokenized_data = self.preprocess_function(test_data)
        return train_tokenized_data, test_tokenized_data

## Обучение модели и сохранение результатов

In [6]:


class TextClassificationModelTrainer:
    def __init__(self, train_data, test_data):
        self.train_data = train_data
        self.test_data = test_data
        self.model_ckpt = config.MODEL_CKPT
        self.id2label = config.ID2LABEL
        self.label2id = config.LABEL2ID
        self.num_labels = len(self.id2label)
        self.device = config.DEVICE
        self.eval_metric = config.EVAL_METRIC
        self.model_out_dir = config.MODEL_OUT_DIR
        self.num_epochs = config.NUM_EPOCHS
        self.lr = config.LR
        self.batch_size = config.BATCH_SIZE
        self.weight_decay = config.WEIGHT_DECAY
        self.eval_strategy = config.EVAL_STRATEGY
        self.save_strategy = config.SAVE_STRATEGY
        self.logging_strategy = config.LOGGING_STRATEGY
        self.push_to_hub = config.PUSH_TO_HUB
        self.model = AutoModelForSequenceClassification.from_pretrained(
                                                                        self.model_ckpt,
                                                                        id2label=self.id2label,
                                                                        label2id=self.label2id,
                                                                        num_labels=self.num_labels
                                                                        ).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_ckpt)
        self.eval_metric_computer = evaluate.load(self.eval_metric)
        self.data_collator = DataCollatorWithPadding(self.tokenizer)

    def compute_metrics(self, eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return self.eval_metric_computer.compute(predictions=predictions, references=labels)

    def set_training_args(self):
        return TrainingArguments(
        output_dir = self.model_out_dir,
        num_train_epochs=self.num_epochs,
        learning_rate = self.lr,
        per_device_train_batch_size = self.batch_size,
        per_device_eval_batch_size = self.batch_size,
        weight_decay = self.weight_decay,
        evaluation_strategy = self.eval_strategy,
        save_strategy = self.save_strategy,
        logging_strategy = self.logging_strategy,
        push_to_hub = self.push_to_hub
        )

    def model_trainer(self):
        return Trainer(
            model = self.model,
            args = self.set_training_args(),
            data_collator = self.data_collator,
            train_dataset = self.train_data,
            eval_dataset = self.test_data,
            compute_metrics = self.compute_metrics
        )
    def train_and_save_and_push_to_hub(self):
      trainer = self.model_trainer()
      trainer.train()
      trainer.push_to_hub()





In [7]:

from huggingface_hub import notebook_login
notebook_login()


In [11]:
if __name__ == "__main__":
    textclassificationdataset = TextClassificationDataset()
    train_data, test_data = textclassificationdataset.gen_classification_dataset()
    textclassificationtrainer = TextClassificationModelTrainer(train_data, test_data)
    textclassificationtrainer.train_and_save_and_push_to_hub()


Map:   0%|          | 0/3450 [00:00<?, ? examples/s]

Map:   0%|          | 0/863 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.596400,0.304811,0.901506
2,0.268700,0.194527,0.943221
3,0.178700,0.177390,0.949015


## Проверка работы модели

In [15]:

classifier = pipeline("sentiment-analysis", model=config.MODEL_OUT_DIR, tokenizer="distilbert-base-uncased")
classifier("колоноскопия")# данное название отсутствовало в данных для обучения и тестирования

[{'label': 'Инструментальная диагностика', 'score': 0.6913766264915466}]

In [16]:
classifier("Осмотр консультация врача физиотерапевта")# данное название присутствовало в данных для обучения и тестирования

[{'label': 'Консультация специалиста', 'score': 0.9962001442909241}]

# Вывод



Эта модель представляет собой доработанную версию distilbert-base-uncased. На оценочном наборе он достигает следующих результатов:

- Loss: 0.1774
- Accuracy: 0.9490


## Training procedure

### Training hyperparameters

Во время обучения использовались следующие гиперпараметры:
- learning_rate: 2e-05
- train_batch_size: 16
- eval_batch_size: 16
- seed: 42
- optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
- lr_scheduler_type: linear
- num_epochs: 3

### Training results

| Training Loss | Epoch | Step | Validation Loss | Accuracy |
|:-------------:|:-----:|:----:|:---------------:|:--------:|
| 0.5964        | 1.0   | 216  | 0.3048          | 0.9015   |
| 0.2687        | 2.0   | 432  | 0.1945          | 0.9432   |
| 0.1787        | 3.0   | 648  | 0.1774          | 0.9490   |


### Framework versions

- Transformers 4.38.2
- Pytorch 2.2.1+cu121
- Datasets 2.18.0
- Tokenizers 0.15.2
